[DuckDB Tutorial - DuckDB course for beginners](https://www.youtube.com/watch?v=AjsB6lM2-zw)

[Introduction](https://marclamberti.com/blog/duckdb-getting-started-for-beginners/)

Date : 27-12-2023

In [1]:
# %%
import pandas as pd
import glob
import time
import duckdb

In [3]:
# Création d'une base de données vide : 
conn = duckdb.connect()

In [12]:
# Instruction avec la librairie pandas :
# Temps d'écoulement de chargement de tous les fichiers .csv du répertoire
# 'dataset'
cur_time = time.time()

df = pd.concat([pd.read_csv(f) 
                for f in glob.glob('dataset/*.csv')])

print(f"time: {(time.time() - cur_time)}")

print(df.head(10))

time: 0.7440307140350342
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1      NaN                         NaN              NaN        NaN   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1             NaN                                     NaN  
2  04/07/19 22:30       

In [13]:
# Instruction avec une requête SQL avec la librairie duckdb :
# Temps d'écoulement de chargement de tous les fichiers .csv du répertoire
# 'dataset' à partir d'une requête SQL convertie en DF pandas
cur_time = time.time()

df = conn.execute("""
SELECT * 
FROM read_csv_auto('dataset/*.csv', header=True)
LIMIT 10
""").df()

print(f"time: {(time.time() - cur_time)}")

print(df)

time: 0.15200042724609375
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1     None                        None             None       None   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1            None                                    None  
2  04/07/19 22:30      

In [14]:
# Requête SQL : Récupération de toutes les lignes des fichiers .csv
df = conn.execute("""
SELECT * 
FROM read_csv_auto('dataset/*.csv', header=True)
""").df()


In [15]:
# Requête SQL : enregistrement virtuel de la base de données dans une 
# table nommée 'df_view'
conn.register("df_view", df)

In [16]:
# Requête SQL : analyse de la table virtuelle enregistrée ci-avant
conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,VARCHAR,YES,None,None,NaN
1,Product,VARCHAR,YES,None,None,NaN
2,Quantity Ordered,VARCHAR,YES,None,None,NaN
3,Price Each,VARCHAR,YES,None,None,NaN
4,Order Date,VARCHAR,YES,None,None,NaN
5,Purchase Address,VARCHAR,YES,None,None,NaN


In [17]:
# Requête SQL : nombre de lignes dans la table virtuelle
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,186850


In [18]:
# Instructions pandas : valeurs nulles dans chaque colonne
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [19]:
# Instruction pandas : suppression des lignes avec uniquement 
# des valeurs nulles
df = df.dropna(how='all')

In [20]:
df.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [21]:
# Requête SQL : nombre de lignes dans la df d'origine
conn.execute("SELECT COUNT(*) FROM df").df()

,count_star()
0,186305


In [22]:
# Requête SQL : affichage de la ligne avec Order ID = 295665
conn.execute("""SELECT * FROM df WHERE "Order ID"='295665';""").df()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"


In [23]:
# Requête SQL : enregistrement des champs de la DF dans une table 'sales'
# TRY_CAST("Order ID" AS INTEGER) NOTNULL : suppression des lignes d'en-tête
# présentes dans la DF
conn.execute("""
CREATE OR REPLACE TABLE sales AS
    SELECT
        "Order ID"::INTEGER AS order_id,
        Product AS product,
        "Quantity Ordered"::INTEGER AS quantity,
        "Price Each"::DECIMAL AS price_each,
        strptime("Order Date", '%m/%d/%Y %H:%M')::DATE as order_date,
        "Purchase Address" AS purchase_adress
    FROM df
    WHERE
        TRY_CAST("Order ID" AS INTEGER) NOTNULL;
""")

In [24]:
# Requête SQL : affichage de certaines colonnes de la table créée ci-avant
conn.execute("""
SELECT *
    EXCLUDE (product, order_date, purchase_adress) FROM sales         
             """).df()

,order_id,quantity,price_each
0,176558,2,11.95
1,176559,1,99.99
2,176560,1,600.00
3,176560,1,11.99
4,176561,1,11.99
...,...,...,...
185945,259353,3,2.99
185946,259354,1,700.00
185947,259355,1,700.00
185948,259356,1,379.99


In [25]:
# Requête SQL : valeur minimum à afficher pour certaines colonnes
conn.execute("""
SELECT
    MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_adress)))
FROM sales         
             """).df()

,min(sales.order_id),min(sales.quantity),min(sales.price_each)
0,141234,1,2.99


In [26]:
# Requête SQL : création d'une nouvelle table virtuelle avec l'agrégation
# de certaines données de la table virtuelle 'sales'
conn.execute("""
CREATE OR REPLACE TABLE aggregated_sales AS
    SELECT
        order_id,
        COUNT(1) as nb_orders,
        MONTH(order_date) as month,
        str_split(purchase_adress, ',')[2] AS city,
        SUM(quantity * price_each) AS revenue
    FROM sales
    GROUP BY ALL;
""")

In [27]:
# Requête SQL : affichage des données de la table virtuelle
# 'aggregated_sales'
conn.execute("FROM aggregated_sales").df()

,order_id,nb_orders,month,city,revenue
0,196937,1,5,Los Angeles,379.99
1,196944,1,5,Los Angeles,149.99
2,196950,1,5,New York City,11.99
3,196951,1,5,San Francisco,150.00
4,196955,1,5,Dallas,379.99
...,...,...,...,...,...
178432,196915,1,5,Atlanta,600.00
178433,196916,1,5,Dallas,2.99
178434,196918,1,5,Dallas,5.98
178435,196920,2,5,Los Angeles,26.90


In [28]:
# Requête SQL : Revenu total par ville par trie décroissant
conn.execute("""
SELECT
    city,
    SUM(revenue) AS total
FROM aggregated_sales
GROUP BY city
ORDER BY total DESC
             """).df()

,city,total
0,San Francisco,8262203.91
1,Los Angeles,5452570.80
2,New York City,4664317.43
3,Boston,3661642.01
4,Atlanta,2795498.58
5,Dallas,2767975.40
6,Seattle,2747755.48
7,Portland,2320490.61
8,Austin,1819581.75


In [29]:
# Requête SQL : exportation sous un type .parquet
conn.execute("""
             COPY (FROM aggregated_sales) 
             TO 'dataset/aggregated_sales.parquet' (FORMAT 'parquet');
             """)

In [30]:
# Requête SQL : récupation du fichier .parquet converti en DF pandas
conn.execute("""
FROM 'dataset/aggregated_sales.parquet'
            """).df()

,order_id,nb_orders,month,city,revenue
0,196937,1,5,Los Angeles,379.99
1,196944,1,5,Los Angeles,149.99
2,196950,1,5,New York City,11.99
3,196951,1,5,San Francisco,150.00
4,196955,1,5,Dallas,379.99
...,...,...,...,...,...
178432,196915,1,5,Atlanta,600.00
178433,196916,1,5,Dallas,2.99
178434,196918,1,5,Dallas,5.98
178435,196920,2,5,Los Angeles,26.90
